<a href="https://colab.research.google.com/github/afijithpv/Python-assignments-/blob/main/Designathon%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Write a program that checks if a given year is a leap year. A leap year is divisible by 4 but not by 100, or it is divisible by 400. For example, 2000 and 2004 are leap years, but 1900 is not.

In [ ]:
from flask import render_template, url_for, flash, redirect, request, session
from app import app, mysql
from werkzeug.security import generate_password_hash, check_password_hash
import MySQLdb.cursors

@app.route('/')
def home():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute('SELECT * FROM recipes')
    recipes = cursor.fetchall()
    return render_template('home.html', recipes=recipes)

@app.route('/recipe/<int:recipe_id>', methods=['GET', 'POST'])
def recipe_detail(recipe_id):
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute('SELECT * FROM recipes WHERE id = %s', (recipe_id,))
    recipe = cursor.fetchone()
    cursor.execute('SELECT comments.comment, users.username, comments.date_posted FROM comments JOIN users ON comments.user_id = users.id WHERE recipe_id = %s', (recipe_id,))
    comments = cursor.fetchall()
    return render_template('recipe_detail.html', recipe=recipe, comments=comments)

@app.route('/submit_recipe', methods=['GET', 'POST'])
def submit_recipe():
    if 'user_id' not in session:
        flash('You need to log in to submit a recipe', 'danger')
        return redirect(url_for('login'))
    if request.method == 'POST':
        name = request.form['name']
        ingredients = request.form['ingredients']
        instructions = request.form['instructions']
        cooking_time = request.form['cooking_time']
        serving_size = request.form['serving_size']
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('INSERT INTO recipes (name, ingredients, instructions, cooking_time, serving_size, user_id) VALUES (%s, %s, %s, %s, %s, %s)',
                       (name, ingredients, instructions, cooking_time, serving_size, session['user_id']))
        mysql.connection.commit()
        flash('Your recipe has been submitted!', 'success')
        return redirect(url_for('home'))
    return render_template('submit_recipe.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = generate_password_hash(request.form['password'])
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('INSERT INTO users (username, email, password) VALUES (%s, %s, %s)', (username, email, password))
        mysql.connection.commit()
        flash('Your account has been created! You can now log in', 'success')
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('SELECT * FROM users WHERE email = %s', (email,))
        user = cursor.fetchone()
        if user and check_password_hash(user['password'], password):
            session['user_id'] = user['id']
            session['username'] = user['username']
            flash('You have been logged in!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Login Unsuccessful. Please check email and password', 'danger')
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('user_id', None)
    session.pop('username', None)
    flash('You have been logged out', 'success')
    return redirect(url_for('home'))


In [ ]:
# List of years to check
years_to_check = [2000, 2004, 1900, 2020, 2100, 2024]

# Iterate through each year and check if it's a leap year
for year in years_to_check:
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        print(year, "is a leap year")
    else:
        print(year, "is not a leap year")

2000 is a leap year
2004 is a leap year
1900 is not a leap year
2020 is a leap year
2100 is not a leap year
2024 is a leap year


2. Write a program that calculates the factorial of a number. The factorial of a non-negative integer n is the product of all positive integers less than or equal to n. For example, the factorial of 5 (denoted as 5!) is 5 x 4 x 3 x 2 x 1 = 120.

In [ ]:
# Input: Enter a non-negative integer for which you want to calculate factorial
number = int(input("Enter a non-negative integer: "))

# Check if the input number is valid (non-negative)
if number < 0:
    print("Factorial is not defined for negative numbers.")
elif number == 0:
    print("Factorial of 0 is 1")
else:
    factorial = 1
    # Calculate factorial using a loop (iterative approach)
    for i in range(1, number + 1):
        factorial *= i

    # Print the factorial of the input number
    print("The factorial of", number, "is",factorial)

Enter a non-negative integer: 5
The factorial of 5 is 120


3. Write a program that generates the Fibonacci sequence up to a certain number of terms. The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones, starting from 0 and 1. For example, the first 10 terms of the Fibonacci sequence are 0, 1, 1, 2, 3, 5, 8, 13, 21, 34.

In [ ]:
# Input: Number of terms in the Fibonacci sequence to generate
num_terms = int(input("Enter the number of Fibonacci terms to generate: "))

# Check if the input number of terms is valid (positive integer)
if num_terms <= 0:
    print("Please enter a positive integer.")
else:
    # Initialize the first two terms of the Fibonacci sequence
    first_term = 0
    second_term = 1

    # Print the first two terms (special case for Fibonacci sequence)
    if num_terms >= 1:
        print(first_term, end=' ')
    if num_terms >= 2:
        print(second_term, end=' ')

    # Generate subsequent terms of the Fibonacci sequence using a loop
    for _ in range(2, num_terms):
        next_term = first_term + second_term
        print(next_term, end=' ')
        # Update the values of first_term and second_term for the next iteration
        first_term = second_term
        second_term = next_term

Enter the number of Fibonacci terms to generate: 5
0 1 1 2 3 


4. Write a program to print all prime numbers within 1 to 1000.

In [ ]:
# Iterate through numbers from 2 to 1000 (inclusive)
for num in range(2, 1001):
    is_prime = True  # Assume num is prime initially

    # Check if num is divisible by any number from 2 to num-1
    for divisor in range(2, num):
        if num % divisor == 0:
            is_prime = False
            break

    # Print the number if it is prime
    if is_prime:
        print(num, end=' ')

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499 503 509 521 523 541 547 557 563 569 571 577 587 593 599 601 607 613 617 619 631 641 643 647 653 659 661 673 677 683 691 701 709 719 727 733 739 743 751 757 761 769 773 787 797 809 811 821 823 827 829 839 853 857 859 863 877 881 883 887 907 911 919 929 937 941 947 953 967 971 977 983 991 997 


5. Write a program to check whether given number is Amstrong
An Armstrong number is a number that is equal to the sum of its own digits raised to the power of the number of digits. For example, 153 is an Armstrong number because:
1^3 + 5^3 + 3^3 = 1 + 125 + 27 = 153

In [ ]:
# Input: Enter a number to check if it is an Armstrong number
number = int(input("Enter a number: "))

# Convert the number to a string to determine the number of digits
num_str = str(number)
num_digits = len(num_str)

# Calculate the sum of digits raised to the power of the number of digits
sum_of_powers = 0
temp = number

while temp > 0:
    digit = temp % 10  # Extract the last digit
    sum_of_powers += digit ** num_digits
    temp //= 10  # Remove the last digit from the number

# Check if the sum of powers is equal to the original number
if sum_of_powers == number:
    print(number," is an Armstrong number")
else:
    print(number, "is not an Armstrong number")

Enter a number: 153
153 is an Armstrong number
